### Instance Properties

So far we have seen instance attributes. In general terms these are simply instance level "variables" that are directly accessible, usually via dot notation, such as `p.name`.

If you're background is Java you've probably learned that you should never use bare attributes in Java classes, and instead implement getter and setter functions.

Let's see how you might replicate this in Python, using a Java-styled approach:

In [1]:
class Person:
    def __init__(self, name):
        self.set_name(name)
        
    def set_name(self, value):
        self.name = value
    
    def get_name(self):
        return self.name

In [4]:
p = Person('Alex')

In [5]:
p.get_name()

'Alex'

In [6]:
p.set_name('Eric')

In [7]:
p.get_name()

'Eric'

Of course, the instance still uses `name` to store the value for `name`:

In [8]:
p.name

'Eric'

And it is still directly accessible:

In [9]:
p.name = 'Alex'

In [10]:
p.name

'Alex'

In [11]:
p.get_name()

'Alex'

In Java, we would make the "backing" variable private. But there is no such thing in Python - we do not have the concept of private scopes. So, instead we use a **convention** that prefixes private variables with an underscore `_`.

This underscore is meant to indicate to users of our class that certain attributes are "private" and should not be directly altered. Of course, anyone can still mess with that "pseudo-private" attribute, but at least fair warning was given, and we're not responsible for their actions!

So let's tweak our class to reflect this convention:

In [12]:
class Person:
    def __init__(self, name):
        self.set_name(name)
        
    def set_name(self, value):
        self._name = value
    
    def get_name(self):
        return self._name

The main reason why Java strongly recommends using accessor methods for instance properties is that although you can very well start off using bare attributes, at some point in the future you may want to implement some sort of validation when a an attribute is being set.

If we started off with this class:

In [13]:
class Person:
    def __init__(self, name):
        self.name = name

Then users would be using our class this way:

In [14]:
p = Person('Alex')
p.name = 'Eric'

Now, if later we want to implement validation, we need to change to using accessor methods like we just did:

In [17]:
class Person:
    def __init__(self, name):
        self.set_name(name)
        
    def set_name(self, value):
        if len(value) < 2:
            raise ValueError('Name must be at least two characters long.')
        self._name = value
    
    def get_name(self):
        return self._name

And now we have **broken the interface** of our class, since any code that used the bare attribute is now broken:

In [18]:
p = Person('Alex')
p.name = 'Eric'

Although it seems like this worked, we ended creating a **new** instance attribute `name`, on top of the one we really want to use, `_name`:

In [19]:
p.name

'Eric'

In [20]:
p.get_name()

'Alex'

So this is why in Java we usually just implement accessor methods right away.

But Python has a different approach that allows us to switch from bare attributes to accessor methods **without** changing the interface of our class.

Let's look at how we can define accessor methods. We'll start with a "read-only" property first that provides a getter accessor method:

In [21]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

We can then use our class this way:

In [22]:
p = Person('Alex')
p.name

'Alex'

Notice how we now have a function used to return the property value for `name`, but we did not have to call a function - we used the usual dot notation syntax `.name` - and the fact that a method was actually called was transparent to us as a user of the class. 

So whether we use:

In [23]:
class Person:
    def __init__(self, name):
        self.name = name

or

In [24]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

The **way** we access `name` is the **same** in both cases.

For this reason Python often uses bare attributes for properties until such time as we need to use accessor methods (maybe to add validation) - and doing so **does not** break the interface.

To create a getter, we simply create a **method** with the name we want for our property, and **decorate** it with `@property`. How decorators work is beyond the scope of this primer, for now we can just use it.

We can also create a setter accessor, also using a decorator. Here we want to define another method, also with the same name as the property name, but decorated slightly differently. 

In order to define a setter, we first need to define the getter, and then use the property name to decorate the setter.

Here's a simple example:

In [25]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value

Notice how we used the **property name** in the setter decorator.

We can then use our class as follows:

In [26]:
p = Person('Alex')

In [27]:
p.name

'Alex'

In [28]:
p.name = 'Eric'

In [29]:
p.name

'Eric'

**Note**: Notice how the property name was repeated several times while creating the getter and setter methods:

```
@property
def name(self):
    return self._name

@name.setter
def name(self, value):
    self._name = value
```

It is imperative that you always do this - for example writing it this way will not work correctly:

```
@property
def name(self):
    return self._name

@name.setter
def set_name(self, value):
    self._name = value
```

Getter methods can also be handy to create calculated properties, i.e. properties that do not have an actual backing variable, but are computed each tiome the property is requested:

In [1]:
class Person:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
    
    @property
    def full_name(self):
        return ', '.join([self.last_name, self.first_name])

In [2]:
p = Person('Isaac', 'Newton')

In [3]:
p.full_name

'Newton, Isaac'

Notice how I left `first_name` and `last_name` as bare attributes. If I later decided to add some validation to these values, I could do so without breaking the class interface:

In [4]:
class Person:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
    
    @property
    def full_name(self):
        return ', '.join([self.last_name, self.first_name])
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError(f'str type expected, got {type(value)} instead.')
        value = value.strip()
        if not value:
            raise ValueError('Cannot be blank')
        self._first_name = value
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        if not isinstance(value, str):
            raise TypeError(f'str type expected, got {type(value)} instead.')
        value = value.strip()
        if not value:
            raise ValueError('Cannot be blank')
        self._last_name = value

Let's try this out:

In [6]:
p = Person(first_name='Isaac', last_name='Newton')

In [7]:
p.full_name

'Newton, Isaac'

In [8]:
p.first_name

'Isaac'

And let's try out the validation:

In [9]:
p = Person('', 'Socrates')

ValueError: Cannot be blank

In [10]:
p = Person(10, 'Socrates')

TypeError: str type expected, got <class 'int'> instead.

And setting these properties after the instance has been created works too:

In [12]:
p = Person('John', 'von Neumann')

In [13]:
p.first_name = None

TypeError: str type expected, got <class 'NoneType'> instead.

In [14]:
p.first_name = "   "

ValueError: Cannot be blank